In [ ]:
# from tensorflow import ConfigProto
# from tensorflow import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model
sess = tf.compat.v1.Session()
graph = tf.compat.v1.get_default_graph()

set_session(sess)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import sys
from tqdm import tqdm, tqdm_notebook
import glob
import shutil
import time      # time.perf_counter()
import random

import matplotlib.pyplot as plt
import cv2

import xml.etree.ElementTree as ET

from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv1D
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.preprocessing import image

from keras.layers import Lambda
from keras.layers import Concatenate
from keras.optimizers import RMSprop
from keras.layers import Multiply
from functools import partial



In [2]:
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.__version__)

Num GPUs Available:  3
2.2.0-rc3


In [3]:
import os
# Create a MirroredStrateg, If Multi-GPU available
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
tf.config.set_soft_device_placement(True)

In [4]:
import keras

In [13]:

global sess
global graph
with graph.as_default():
    set_session(sess)

    root_dir='/mnt/HDD02/WGAN/old_backup/110Signs/'
    out_dir = '/mnt/HDD02/WGAN/old_backup/110Signs/GAN/'
    #sub_dir=['Pick','Kneel','Crawl','Sit','Short','Scissor','Limp','Toes']
    #sub_dir=['Towards','Away','Crawl']
    #sub_dir=['You','Hello','Walk','Drink','Friend','Knife','Well','Car','Engineer','Mountain','Lawyer','Hospital','Health','Earthquake','Breath','Help','Push','Go','Come','Write']
#     sub_dir=['Ben','B_comeUp','Ben_chair','FalSta','SitSilSta','StaWlk','WlkBen','WlkBenWlk','WlkFalSta','WlkKne','WlkKneSta']
    sub_dir=['1','2','3','4'.'5','6','7','8','9','10']
    words=len(sub_dir)

    for i in range (0,len(sub_dir)):
        kernel_start = time.perf_counter()
        kernel_time_limit = 60*20    #### running time

        # image size
        img_size = 128
        channels = 3
        img_shape = (img_size, img_size, channels)    # (64,64,3)

        # z(latent variable) size
        z_dim = 100
        z_shape = (z_dim,)

        # gradient penalty coefficient "λ"
        penaltyLambda = 10    # d_loss = f_loss - r_loss + λ･penalty

        # critic(discriminator) iterations per generator iteration
        trainRatio = 5

        batch_size = 2       # 16 or 64 better?
        rec_interval = 2000



        DIRanno= root_dir + sub_dir[i] +'/'
        DIRimg = DIRanno
        DIRout = out_dir + sub_dir[i] +'/'

        def loadImage(fPath, resize = True):
            img = cv2.imread(fPath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      # BGR to RGB
            if resize:
                xmin,ymin,xmax,ymax = clipImage(fPath)      # clip to square
                if xmin >= 0:                               # exist Annotation
                    img = img[ymin:ymax, xmin:xmax, :]      # [h,w,c]
                # Interpolation method
                if xmax - xmin > img_size:
                    interpolation = cv2.INTER_AREA          # shrink
                else:
                    interpolation = cv2.INTER_CUBIC         # expantion
                img = cv2.resize(img, (img_size, img_size),
                            interpolation = interpolation)  # resize
            return img

        def clipImage(fPath):
            imgName = os.path.basename(fPath)[:-4].split("_")
            breed = imgName[0]
            dog = imgName[1]
            path = glob.glob(os.path.join(DIRanno, breed + "*", breed +"_" + dog))
            if len(path) > 0:
                tree = ET.parse(path[0])
                root = tree.getroot()    # get <annotation>
                size = root.find('size')
                width = int(size.find('width').text)
                height = int(size.find('height').text)
        #        objects = root.findall('object')   # ToDo: correspond multi objects
        #        for object in objects:
                object = root.find('object')
                bndbox = object.find('bndbox') 
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                xmin = max(0, xmin - 4)        # 4 : margin
                xmax = min(width, xmax + 4)
                ymin = max(0, ymin - 4)
                ymax = min(height, ymax + 4)

                w = max(xmax - xmin, ymax - ymin, img_size)   # ideal w
                w = min(w, width, height)                     # available w

                if w > xmax - xmin:
                    xmin = min(max(0, xmin - int((w - (xmax - xmin))/2)), width - w)
                    xmax = xmin + w
                if w > ymax - ymin:
                    ymin = min(max(0, ymin - int((w - (ymax - ymin))/2)), height - w)
                    ymax = ymin + w

            else:
                xmin = -1; ymin = -1; xmax = -1; ymax = -1;     # no annotation ?

            return xmin,ymin,xmax,ymax

        # train data
        all_fNames = os.listdir(DIRimg)
        x_train = np.zeros((len(all_fNames),img_size,img_size,3))
        for i in tqdm(range(len(all_fNames))):
            path = os.path.join(DIRimg, all_fNames[i])
            x_train[i] = loadImage(path)

        x_train = x_train / 255.
        print(x_train.shape)



        def build_generator():
            input = Input(shape=z_shape)
            x = Dense(8*img_size*img_size, activation="relu")(input)
            x = Reshape((img_size//8, img_size//8, -1))(x)
            x = UpSampling2D((2, 2))(x)
            x = Conv2D(64*4, kernel_size=3, strides=1, padding="same",
                           use_bias=False)(x)
            x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x, training=1)
            x = Conv2D(64*4, kernel_size=3, strides=1, padding="same", use_bias=False)(x)
            x = BatchNormalization(momentum=0.9, epsilon=1e-5, )(x, training=1)
            x = Activation("relu")(x)
            x = Conv2D(64*4, kernel_size=3, strides=1, padding="same", use_bias=False)(x)
            x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x, training=1)
            x = Activation("relu")(x)
            x = UpSampling2D((2, 2))(x)
            x = Conv2D(64*2, kernel_size=3, strides=1, padding="same",
                       use_bias=False)(x)
            x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x, training=1)
            x = Conv2D(64*2, kernel_size=3, strides=1, padding="same", use_bias=False)(x)
            x = BatchNormalization(momentum=0.9, epsilon=1e-5, )(x, training=1)
            x = Activation("relu")(x)
            x = Conv2D(64*2, kernel_size=3, strides=1, padding="same", use_bias=False)(x)
            x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x, training=1)
            x = Activation("relu")(x)
            x = UpSampling2D((2, 2))(x)
            x = Conv2D(64*1, kernel_size=3, strides=1, padding="same",
                           use_bias=False)(x)
            x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x,training=1)
            x = Conv2D(64*1, kernel_size=3, strides=1, padding="same", use_bias=False)(x)
            x = BatchNormalization(momentum=0.9, epsilon=1e-5, )(x, training=1)
            x = Activation("relu")(x)
            x = Conv2D(64*1, kernel_size=3, strides=1, padding="same", use_bias=False)(x)
            x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x, training=1)
            x = Activation("relu")(x)
            x = Conv2D(3, kernel_size=3, strides=1, padding="same", activation="tanh",
                       use_bias=False,)(x)

            model = Model(input, x)
            print("●generator")
            model.summary()
            return model


        def build_discriminator():
            input = Input(shape=img_shape)


            #img = Input(shape=img_shape)


            der=Lambda(lambda x:x[:,:,:,:1])(input) # one channel only
            print(der)
            Re=Lambda(lambda x:tf.keras.backend.squeeze(x,axis=-1))(der)
            print('Re',Re)  
            x2=Lambda(lambda x:K.greater(x,0.0589))(Re) 
            x22=Lambda(lambda x:K.cast(x,tf.float32))(x2)
            print('X22',x22) 

            x3=Multiply()([Re,x22])
            print('x3',x3) 

            Q=Lambda(lambda x:K.sum(x,axis=1))(x3)
            print('Q',Q)
            Q1=Lambda(lambda x:x*0.97)(Q)
            DF=Lambda(lambda x:tf.unstack(x,axis=-1))(x3)
            DFG=[]
            zw=Lambda(lambda x:tf.unstack(x,axis=-1))(Q1)
            for t,s in zip(DF,zw):
                P=Lambda(lambda x:tf.keras.backend.reverse(x,axes=1))(t)

                P3=Lambda(lambda x:tf.math.cumsum(x,axis=1))(P)
                PW=Lambda(lambda x: tf.expand_dims(x,axis=-1)) (s)

                WE=Lambda(lambda x: tf.tile(x,[1,128])) (PW)
                subtracted = keras.layers.Subtract()([P3, WE])
                #S=Lambda(lambda x:K.greater(x,0.0))(P3-WE) 
                S=Lambda(lambda x:K.greater(x,0.0))(subtracted) 

                R=Lambda(lambda x:tf.keras.backend.cast(x,tf.float32))(S)
                idx=Lambda(lambda x:128-tf.keras.backend.argmax(x))(R)
                DFG.append(idx)
            B12=Lambda(lambda x:K.stack(x,axis=1))(DFG)
            print('p',P)
            print('WE',WE)
            print('R',R)

            B12=Lambda(lambda x:K.cast(x,tf.float32))(B12)
            print('B12',B12)
            B13=Dense(128)(B12)
            print('B13',B13)
            D=Lambda(lambda x: tf.expand_dims(x,axis=1)) (B12)
            print(D.shape)
            D1=Conv1D(64,1)(D)
            print(D1.shape)
            D1=Conv1D(32,1)(D1)
            print(D1.shape)
            D1=BatchNormalization()(D1, training=1)
            D1=Conv1D(32,1)(D1)
            D1=BatchNormalization()(D1, training=1)
            print('D1',D1)
            D1= Lambda(lambda x:tf.keras.backend.squeeze(x,axis=1))(D1)
            D1=Dense(128)(D1)

            print('D11',D1)


    #         D1=Reshape([1,1,128])(D1)
    #         print('e',D1)
    #         D=Lambda(lambda x: tf.expand_dims(x,axis=1)) (D)
    #         print('f',D)
    #         E=UpSampling2D(size=(2, 2))(D1)
    #     # #     E1=batchnorm()(E, training=1)
    #         F=UpSampling2D(size=(8, 8))(E)
    #     # #     F1=batchnorm()(F, training=1)
    #         F2= LeakyReLU(alpha=0.2)(F)
    #         F3 = ZeroPadding2D(1)(F2)
    #         #F4= Dense(64)(F3)



            x1 = Conv2D(64, kernel_size=4, strides=2, padding="same", use_bias=False)(input)
            x1 = LeakyReLU(0.2)(x1)
            x1 = Conv2D(128, kernel_size=4, strides=2, padding="same", use_bias=False)(x1)
            x1 = LeakyReLU(0.2)(x1)
            x1 = Conv2D(256, kernel_size=4, strides=2, padding="same", use_bias=False)(x1)
            x1 = Conv2D(256, kernel_size=3, strides=1, padding="same", use_bias=False)(x1)
            x1 = LeakyReLU(0.2)(x1)
            x1 = Conv2D(256, kernel_size=3, strides=1, padding="same", use_bias=False)(x1)
            x1 = LeakyReLU(0.2)(x1)
            x1 = Conv2D(512, kernel_size=4, strides=2, padding="same", use_bias=False)(x1)
            x1 = LeakyReLU(0.2)(x1)
            x1 = Conv2D(1, kernel_size=1, strides=1, padding="same", use_bias=False)(x1)
            x1 = Flatten()(x1)

            x1 = Concatenate()([x1, D1])

            x1 = Dense(units=1, activation=None)(x1)   # activation = None

            model = Model(input, x1)
            print("●discriminator")
            model.summary()
            return model


        def build_WGANgp(generator, discriminator):
            #### model
            # generator image(fake image)
            z = Input(shape=z_shape)
            f_img = generator(z)
            f_out = discriminator(f_img)
            # real image
            r_img = Input(shape=img_shape)
            r_out = discriminator(r_img)
            print(r_out.shape)
            # average image
            epsilon = K.placeholder(shape=(None,1,1,1))
            a_img = Input(shape=(img_shape),
                          tensor = epsilon * r_img + (1-epsilon) * f_img)
            a_out = discriminator(a_img)

            #### loss
            # original critic(discriminator) loss
            r_loss = K.mean(r_out)
            f_loss = K.mean(f_out)
            # gradient penalty  <this is point of WGAN-gp>
            grad_mixed = K.gradients(a_out, [a_img])[0]
    #         grad_mixed = K.GradientTape(a_out, [a_img])[0]
            norm_grad_mixed = K.sqrt(K.sum(K.square(grad_mixed), axis=[1,2,3]))
            grad_penalty = K.mean(K.square(norm_grad_mixed -1))
            penalty = penaltyLambda * grad_penalty
            # d loss
            d_loss = f_loss - r_loss + penalty +2

            #### discriminator update function
            d_updates = Adam(lr=1e-4, beta_1=0.5, beta_2=0.9). \
                        get_updates(discriminator.trainable_weights,[],d_loss)

            d_train = K.function([r_img, z, epsilon],
                                 [r_loss, f_loss, penalty, d_loss],
                                 d_updates)

            #### generator update function
            g_loss = -1. * f_loss
            g_updates = Adam(lr=1e-4, beta_1=0.5, beta_2=0.9). \
                        get_updates(generator.trainable_weights,[],g_loss)
            g_train = K.function([z], [g_loss], g_updates)

            return g_train, d_train

        # generator Model
        generator = build_generator()
        # discriminator Model
        discriminator = build_discriminator()
        # WGAN-gp Training Model
        G_train, D_train = build_WGANgp(generator, discriminator)

        # fixed z for confirmation of generated image
        z_fix = np.random.normal(0, 1, (64, z_dim)) 

        # list for store learning progress data
        g_loss_list = []
        r_loss_list = []
        f_loss_list = []
        f_r_loss_list = []
        penalty_list = []
        d_loss_list = []

        # (0～1) → (-1～+1)
        X_train = (x_train.astype(np.float32) - 0.5) / 0.5

        def sumple_images(imgs, rows=2, cols=2, figsize=(12,10)):
            fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=figsize)
            for indx, axis in enumerate(axes.flatten()):
                img = image.array_to_img(imgs[indx])    # ndarray → PIL
                imgplot = axis.imshow(img)
                axis.set_axis_off()
            plt.tight_layout()



        iteration = 0
        while time.perf_counter() - kernel_start < kernel_time_limit:

            #### Discriminator
            for j in range(trainRatio):
                # Generator in
                z = np.random.normal(0, 1, (batch_size, z_dim))
                # Generator out Images
                f_imgs = generator.predict(z)
                # Real Images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                r_imgs = X_train[idx]
                # train the discriminator
                epsilon = np.random.uniform(size = (batch_size, 1,1,1))
                r_loss, f_loss, penalty, d_loss = D_train([r_imgs, z, epsilon])

            #### Generator
            # Generator in
            z = np.random.normal(0, 1, (batch_size, z_dim))
            # train the generator
            g_loss = G_train([z])

            #### Record of learning progress
            # loss
            r_loss_list.append(r_loss)
            f_loss_list.append(f_loss)
            f_r_loss_list.append(f_loss - r_loss)
            penalty_list.append(penalty)
            d_loss_list.append(d_loss)
            # generated image sumple
            if (iteration in [100, 1000]) or (iteration % rec_interval == 0):

                print(f'iteration:{iteration} / d_loss:{d_loss:.3f} / g_loss:{sum(g_loss)/len(g_loss):.3f}')
                g_imgs = generator.predict(z_fix)
                imgs = g_imgs * 127.5 + 127.5
                sumple_images(imgs, rows=1, cols=7)
                plt.show()

            iteration += 1

        print("last iteration:",iteration - 1)


        if os.path.exists(DIRout):
            shutil.rmtree(DIRout)
        if not os.path.exists(DIRout):
            os.mkdir(DIRout)


            # generate images for submit
        n = 500
        batch = 64
        for i in tqdm(range(0, n, batch)):
            z = np.random.normal(0,1,size=(batch, z_dim))
            g_imgs = generator.predict(z)
            imgs = g_imgs * 127.5 + 127.5
            for j in range(batch):
                img = image.array_to_img(imgs[j])      # ndarray → PIL 
                img.save(os.path.join(DIRout, 'image_' + str(i+j+1).zfill(5) + '.png'))
                if i+j+1 == n:
                    break
        print(len(os.listdir(DIRout)))


100%|██████████| 36/36 [00:00<00:00, 1994.68it/s]


(36, 128, 128, 3)
●generator
Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, 100)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 131072)            13238272  
_________________________________________________________________
reshape_8 (Reshape)          (None, 16, 16, 512)       0         
_________________________________________________________________
up_sampling2d_22 (UpSampling (None, 32, 32, 512)       0         
_________________________________________________________________
conv2d_113 (Conv2D)          (None, 32, 32, 256)       1179648   
_________________________________________________________________
batch_normalization_76 (Batc (None, 32, 32, 256)       1024      
_________________________________________________________________
conv2d_114 (Conv2D)          

lambda_5610 (Lambda)            (None,)              0           lambda_5609[0][0]                
__________________________________________________________________________________________________
lambda_5617 (Lambda)            (None,)              0           lambda_5616[0][0]                
__________________________________________________________________________________________________
lambda_5624 (Lambda)            (None,)              0           lambda_5623[0][0]                
__________________________________________________________________________________________________
lambda_5631 (Lambda)            (None,)              0           lambda_5630[0][0]                
__________________________________________________________________________________________________
lambda_5638 (Lambda)            (None,)              0           lambda_5637[0][0]                
__________________________________________________________________________________________________
lambda_564

KeyboardInterrupt: 

In [ ]:

root_dir='/mnt/HDD02/WGAN/old_backup/native/'
out_dir = '/mnt/HDD02/WGAN/old_backup/Generated_color_data/'
sub_dir=['Go','Health','Hello','Help','Hospital','Knife','Lawyer','Mountain','Push','Walk','Well','Write','You']
words=len(sub_dir)

for i in range (0,len(sub_dir)):
    DIRanno= root_dir + sub_dir[i] +'/'
    DIRimg = DIRanno
    DIRout = out_dir + sub_dir[i] +'/'
    print(DIRout)

In [ ]:
    n = 500
    batch = 64
    for i in tqdm(range(0, n, batch)):
        z = np.random.normal(0,1,size=(batch, z_dim))
        g_imgs = generator.predict(z)
        imgs = g_imgs * 127.5 + 127.5
        for j in range(batch):
            img = image.array_to_img(imgs[j])      # ndarray → PIL 
            img.save(os.path.join(DIRout, 'image_' + str(i+j+1).zfill(5) + '.png'))
            if i+j+1 == n:
                break
    print(len(os.listdir(DIRout)))
